In [28]:
# import io
# from IPython.nbformat import current

# def execute_notebook(nbfile):
    
#     with io.open(nbfile) as f:
#         nb = current.read(f, 'json')
    
#     ip = get_ipython()
    
#     for cell in nb.worksheets[0].cells:
#         if cell.cell_type != 'code':
#             continue
#         ip.run_cell(cell.input)
#     return

execute_notebook("Euler_Tools.ipynb")



### Problem 21

Let d(n) be defined as the sum of proper divisors of n (numbers less than n which divide evenly into n).
If d(a) = b and d(b) = a, where a ≠ b, then a and b are an amicable pair and each of a and b are called amicable numbers.

For example, the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55 and 110; therefore d(220) = 284. The proper divisors of 284 are 1, 2, 4, 71 and 142; so d(284) = 220.

Evaluate the sum of all the amicable numbers under 10000.

In [23]:
from itertools import combinations

def getExpandedFactors(n, primes=None):
    factors = []
    [factors.extend([x[0]]*x[1]) for x in factorize(n, primes)]
    return factors

def genDivisors(num, primes=None):
    if num > 0:
        yield 1
    factors = getExpandedFactors(num, primes)
    for ix in xrange(1, len(factors)):
        for terms in combinations(factors, ix):
            yield product(terms)

sumDivs = {}

for num in xrange(1, 10001):
    sumDivs[num] = sum(set(genDivisors(num)))

amicables = []
    
for key, val in sumDivs.items():
    if (key == val) or (val < key):
        continue
    if sumDivs.get(val) == key:
        amicables.extend([key, val])

print amicables
print sum(amicables)

[220, 284, 1184, 1210, 2620, 2924, 5020, 5564, 6232, 6368]
31626


### Problem 22

Using names.txt (right click and 'Save Link/Target As...'), a 46K text file containing over five-thousand first names, begin by sorting it into alphabetical order. Then working out the alphabetical value for each name, multiply this value by its alphabetical position in the list to obtain a name score.

For example, when the list is sorted into alphabetical order, COLIN, which is worth 3 + 15 + 12 + 9 + 14 = 53, is the 938th name in the list. So, COLIN would obtain a score of 938 × 53 = 49714.

What is the total of all the name scores in the file?

In [12]:
# import os
# os.system("wget https://projecteuler.net/project/resources/p022_names.txt\
# --no-check-certificate")

def getScore(name, index):
    ordbase = ord('a') - 1
    return index * sum((ord(c) - ordbase) for c in name)

names = (x.strip().lower() for x in open('p022_names.txt').read().replace('"',
                                      '').split(','))
print sum(getScore(cc, ix + 1) for ix, cc in enumerate(sorted(names)))


871198282


### Problem 23

A perfect number is a number for which the sum of its proper divisors is exactly equal to the number. For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.

A number n is called deficient if the sum of its proper divisors is less than n and it is called abundant if this sum exceeds n.

As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24. By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers. However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.

Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.

In [33]:
def sumOfDivisors(num, primes=None):
    return sum(set(genDivisors(num, primes)))

def isAbundant(num, primes=None):
    return True if sumOfDivisors(num, primes) > num else False

def isPerfect(num, primes=None):
    return True if sumOfDivisors(num, primes) == num else False

@timeit
def solve():
    primes = list(genPrimes(29000))
    abundants = [x for x in xrange(2, 28124) if isAbundant(x, primes)]
    labu = len(abundants)
    pairSums = set((abundants[i] + abundants[j]) for i in xrange(labu)
                           for j in xrange(i, labu))
    
    pairSums = sum(x for x in pairSums if x < 28124)

    print (28123 * 28124 / 2) - pairSums
    

solve()

4179871
Time = 6.51341795921


### Problem 24

A permutation is an ordered arrangement of objects. For example, 3124 is one possible permutation of the digits 1, 2, 3 and 4. If all of the permutations are listed numerically or alphabetically, we call it lexicographic order. The lexicographic permutations of 0, 1 and 2 are:

012   021   102   120   201   210

What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5, 6, 7, 8 and 9?

In [42]:
def findSmallestHigherIndex(inp, index):
    prev = inp[index - 1]
    remaining = inp[index:]
    minx = remaining[0]
    minix = index
    for ix, x in enumerate(remaining):
        if (x > prev) and (x < minx):
            minx = x
            minix = ix + index
    return minix
    

def genPermutations(inp):
    inp = sorted(inp)
    yield inp
    # read numbers from right to left
    # if increasing, continue
    #     if reach beginning, return
    # if decreasing, exchange lesser number with smallest higher number on right
    # sort all numbers on the right,
    # yield permutation
    # back to first step
    linp = len(inp)
    while True:
        for index in xrange(linp - 1, -1, -1):
            if index == 0:
                return
            if inp[index - 1] > inp[index]:
                continue
            else:
                idx = findSmallestHigherIndex(inp, index)
                inp[index - 1], inp[idx] = inp[idx], inp[index - 1]
                inp[index:] = sorted(inp[index:])
                yield inp
                break
    return

@timeit
def solve():
    for ix, x in enumerate(genPermutations(range(10))):
        if ix == 999999:
            print "".join(str(y) for y in x)
            break
            
solve()

2783915460
Time = 2.55698108673


### Problem 25

The Fibonacci sequence is defined by the recurrence relation:

Fn = Fn−1 + Fn−2, where F1 = 1 and F2 = 1.
Hence the first 12 terms will be:

```
F1 = 1
F2 = 1
F3 = 2
F4 = 3
F5 = 5
F6 = 8
F7 = 13
F8 = 21
F9 = 34
F10 = 55
F11 = 89
F12 = 144
```
The 12th term, F12, is the first term to contain three digits.

What is the index of the first term in the Fibonacci sequence to contain 1000 digits?

In [44]:
def genFibonacci():
    a, b = 1, 1
    yield 1
    yield 1
    while True:
        a, b = b, a + b
        yield b

@timeit
def solve():
    for ix, x in enumerate(genFibonacci()):
        if len(str(x)) >= 1000:
            print ix + 1
            break

solve()


4782
Time = 0.0582852363586


### Problem 26

A unit fraction contains 1 in the numerator. The decimal representation of the unit fractions with denominators 2 to 10 are given:
```
1/2	= 	0.5
1/3	= 	0.(3)
1/4	= 	0.25
1/5	= 	0.2
1/6	= 	0.1(6)
1/7	= 	0.(142857)
1/8	= 	0.125
1/9	= 	0.(1)
1/10	= 	0.1
```
Where 0.1(6) means 0.166666..., and has a 1-digit recurring cycle. It can be seen that 1/7 has a 6-digit recurring cycle.

Find the value of d < 1000 for which 1/d contains the longest recurring cycle in its decimal fraction part.

In [54]:
def seqLength(num):
    rems = [1]
    while True:
        nxt = (rems[-1] * 10) % num
        if nxt in rems:
            return len(rems) - rems.index(nxt) 
        else:
            rems.append(nxt)
    return

result = sorted(((seqLength(x), x) for x in xrange(2, 1000)), reverse=True)[:10]

print result
print result[0][1]

[(982, 983), (976, 977), (970, 971), (952, 953), (940, 941), (936, 937), (886, 887), (862, 863), (856, 857), (822, 823)]
983
